In [1]:
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import os

import string

import panel as pn
import pandas as pd

from panel.interact import interact, interactive, fixed, interact_manual
from panel import widgets

import plotly.graph_objects as go
from plotly.subplots import make_subplots

pn.extension('plotly')

from pytrends.request import TrendReq

In [2]:
load_dotenv()

True

In [3]:
# Set Alpaca API key and secret
alpaca_key = os.getenv('ALPACA_API_KEY')
alpaca_secret = os.getenv('ALPACA_API_SECRET')

# Create the Alpaca API object
alpaca = tradeapi.REST(alpaca_key, alpaca_secret, api_version = "v2")
timeframe = "1D"
start = pd.Timestamp('2016-05-26', tz = 'US/Pacific').isoformat()
end = pd.Timestamp('2021-06-6', tz = 'US/Pacific').isoformat()


In [4]:
def api_call(x):
    x = x.upper()
    df = alpaca.get_barset(
        x,
        timeframe,
        start = start,
        end = end,
        limit = 1000
    ).df
    ret_df = df[x]['close'].pct_change()
    ret_df.dropna(inplace = True)
    cum_ret = (1 + ret_df).cumprod()
    return cum_ret.head()

In [12]:
interact(api_call, x='AAPL')

Column
    [0] Column
        [0] TextInput(name='x', value='AAPL')
    [1] Row
        [0] DataFrame(Series, name='interactive00182')

In [6]:
sectors = [
    'Communications',
    'Consumer Discretionary',
    'Consumer Staples',
    'Energy',
    'Financial',
    'Healthcare',
    'Industrial',
    'Information Technology',
    'Materials',
    'Real Estate',
    'Utilities'
]

sector_tickers = {
    'Communications':
        {'Min': 'VZ', 'Max': 'LYV', 'Median': 'TMUS', 'Mutual Fund': 'VOX'},
    'Consumer Discretionary':
        {'Min': 'NVR', 'Max': 'F', 'Median': 'HLT', 'Mutual Fund': 'VCR'},
    'Consumer Staples':
        {'Min': 'CLX', 'Max': 'SYY', 'Median': 'PM', 'Mutual Fund': 'VDC'},
    'Energy':
        {'Min': 'COG', 'Max': 'OXY', 'Median': 'SLB', 'Mutual Fund': 'VDE'},
    'Financial':
        {'Min': 'CBOE', 'Max': 'LNC', 'Median': 'BAC', 'Mutual Fund': 'VFH'},
    'Healthcare':
        {'Min': 'DGX', 'Max': 'ALGN', 'Median': 'CAH', 'Mutual Fund': 'VHT'},
    'Industrial':
        {'Min': 'DGX', 'Max': 'TDG', 'Median': 'DE', 'Mutual Fund': 'VIS'},
    'Information Technology':
        {'Min': 'ORCL', 'Max': 'ENPH', 'Median': 'NTAP', 'Mutual Fund': 'VGT'},
    'Materials':
        {'Min': 'NEM', 'Max': 'FCX', 'Median': 'AVY', 'Mutual Fund': 'VAW'},
    'Real Estate':
        {'Min': 'PSA', 'Max': 'SPG', 'Median': 'UDR', 'Mutual Fund': 'VNQ'},
    'Utilities':
        {'Min': 'ED', 'Max': 'AES', 'Median': 'SRE', 'Mutual Fund': 'VPU'}
}

In [7]:
pytrend = TrendReq()

In [8]:
def candlestick(x, y):
    ticker = sector_tickers[x][y]
    file = '../Data/{}.csv'.format(ticker)
    df = pd.read_csv(file, infer_datetime_format=True, parse_dates=True, index_col='Date')
    fig = go.Figure(data=[go.Candlestick(
        x = df.index,
        open = df['Open'],
        high = df['High'],
        low = df['Low'],
        close = df['Close']
    )])
    fig.update_layout(title = ticker, width=1200, height=600)
    return fig

def v_spy(x, y):
    ticker = sector_tickers[x][y]
    file = '../Data/{}.csv'.format(ticker)
    df = pd.read_csv(file, infer_datetime_format=True, parse_dates=True, index_col='Date')
    spy = pd.read_csv('../Data/SPY.csv', infer_datetime_format=True, parse_dates=True, index_col='Date')
    fig = make_subplots()

    trace1 = go.Scatter(
        x = df.index,
        y = df['Close'],
        mode = 'lines',
        name = ticker)

    trace2 = go.Scatter(
        x = spy.index,
        y = spy['Close'],
        mode = 'lines',
        name = 'SPY')
    
    
    fig.add_trace(trace1)
    fig.add_trace(trace2)
    fig.update_yaxes(range=[0, 700])
    fig.update_layout(title = ticker + " versus SPY", width=1200, height=600)
    return fig

def trend(x, y, z):
    ticker = sector_tickers[x][y]
    file = '../Data/{}.csv'.format(ticker)
    pytrend.build_payload(kw_list=[ticker], timeframe='today 5-y')
    trend_df = pytrend.interest_over_time().rename_axis('Date')
    trend_df.index = pd.to_datetime(trend_df.index)
    df = pd.read_csv(file, infer_datetime_format=True, parse_dates=True, index_col='Date')
    overlay = pd.merge(trend_df, df[z], how = 'outer', left_index = True, right_index=True)
    overlay = overlay.loc['2020-06-05':]
    overlay.fillna(method = 'ffill', inplace = True)

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    trace1 = go.Scatter(
        x = overlay.index,
        y = overlay[z],
        mode = 'lines',
        name = z)

    trace2 = go.Scatter(
        x = overlay.index,
        y = overlay[ticker],
        mode = 'lines',
        name = 'Search Interest')

    fig.add_trace(trace1, secondary_y=False)
    fig.add_trace(trace2, secondary_y=True)
    fig.update_yaxes(range=[overlay[z].min()-(overlay[z].std()*.2),overlay[z].max()+(overlay[z].std()*.2)], secondary_y=False)
    fig.update_yaxes(range=[0,100], secondary_y=True)
    fig.update_layout(title = ticker + " Closing Price vs Search Interest", width=1200, height=600)
    
    return fig

def combo(x, y):
    ticker = sector_tickers[x][y]
    file = '../Data/{}.csv'.format(ticker)
    pytrend.build_payload(kw_list=[ticker], timeframe='today 5-y')
    trend_df = pytrend.interest_over_time().rename_axis('Date')
    trend_df.index = pd.to_datetime(trend_df.index)
    df = pd.read_csv(file, infer_datetime_format=True, parse_dates=True, index_col='Date')
    overlay = pd.merge(trend_df, df['Volume'], how = 'outer', left_index = True, right_index=True)
    overlay = overlay.loc['2020-06-05':]
    overlay.fillna(method = 'ffill', inplace = True)
    
    fig = make_subplots(
    rows=2, cols=1,
    specs=[[{"type": "candlestick"}], [{"secondary_y": True}]],
    vertical_spacing=0.25,
    subplot_titles=(ticker, 'Volume & Search Interest'))
    y2 = go.YAxis(range=[overlay['Volume'].min()-(overlay['Volume'].std()*.2),overlay['Volume'].max()+(overlay['Volume'].std()*.2)])
    y3 = go.YAxis(range=[0,100])
    fig.add_trace(go.Candlestick(
        x = df.index,
        open = df['Open'],
        high = df['High'],
        low = df['Low'],
        close = df['Close']
        ), row=1, col=1, secondary_y=False)
    fig.add_trace(go.Scatter(
        x = overlay.index,
        y = overlay['Volume'],
        mode = 'lines',
        name = 'Volume',
        yaxis = 'y2'),
                  row=2,col=1, secondary_y=False)
    fig.add_trace(go.Scatter(
        x = overlay.index,
        y = overlay[ticker],
        mode = 'lines',
        name = 'Search Interest',
        yaxis='y3'),
                  row=2,col=1, secondary_y=True)
    fig.update_layout(width=1200, height=1200)

    return fig

In [9]:
#interact(candlestick, x=sectors, y=['Min', 'Max', 'Median', 'Mutual Fund'])   # works
#interact(v_spy, x = sectors, y=['Min', 'Max', 'Median', 'Mutual Fund'])       # works
#interact(trend, x=sectors, y=['Min', 'Max', 'Median', 'Mutual Fund'], z=['Close', 'Volume'])         # works
candle_pn = interact(candlestick, x=sectors, y=['Min', 'Max', 'Median', 'Mutual Fund'])
spy_pn = interact(v_spy, x = sectors, y=['Min', 'Max', 'Median', 'Mutual Fund'])
trend_pn = interact(trend, x=sectors, y=['Min', 'Max', 'Median', 'Mutual Fund'], z=['Close', 'Volume'])
combo_pn = interact(combo, x = sectors, y=['Min', 'Max', 'Median', 'Mutual Fund'])

/home/wh/.local/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:572: DeprecationWarning:

plotly.graph_objs.YAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.YAxis
  - plotly.graph_objs.layout.scene.YAxis




In [10]:
dashboard = pn.Tabs(
    pn.Row("HELLO"),
    pn.Row(interact(candlestick, x=sectors, y=['Min', 'Max', 'Median', 'Mutual Fund'])),
    pn.Row(interact(v_spy, x = sectors, y=['Min', 'Max', 'Median', 'Mutual Fund'])),
    pn.Row(interact(trend, x=sectors, y=['Min', 'Max', 'Median', 'Mutual Fund'], z=['Close', 'Volume'])),
    pn.Column(pn.Row(candle_pn[0][0], candle_pn[0][1]), candle_pn[1]),
    pn.Column(pn.Row(spy_pn[0], spy_pn[1])),
    pn.Column(pn.Row(combo_pn[0][0], combo_pn[0][1]), combo_pn[1]))

In [11]:
dashboard.servable()

Tabs
    [0] Row
        [0] Markdown(str)
    [1] Row
        [0] Column
            [0] Column
                [0] Select(name='x', options=['Communications', ...], value='Communications')
                [1] Select(name='y', options=['Min', 'Max', ...], value='Min')
            [1] Row
                [0] Plotly(Figure, name='interactive00145')
    [2] Row
        [0] Column
            [0] Column
                [0] Select(name='x', options=['Communications', ...], value='Communications')
                [1] Select(name='y', options=['Min', 'Max', ...], value='Min')
            [1] Row
                [0] Plotly(Figure, name='interactive00154')
    [3] Row
        [0] Column
            [0] Column
                [0] Select(name='x', options=['Communications', ...], value='Communications')
                [1] Select(name='y', options=['Min', 'Max', ...], value='Min')
                [2] Select(name='z', options=['Close', 'Volume'], value='Close')
            [1] Row
                [0] Plotly(Figure, name='interactive00163')
    [4] Column
        [0] Row
            [0] Select(name='x', options=['Communications', ...], value='Communications')
            [1] Select(name='y', options=['Min', 'Max', ...], value='Min')
        [1] Row
            [0] Plotly(Figure, name='interactive00109')
    [5] Column
        [0] Row
            [0] Column
                [0] Select(name='x', options=['Communications', ...], value='Communications')
                [1] Select(name='y', options=['Min', 'Max', ...], value='Min')
            [1] Row
                [0] Plotly(Figure, name='interactive00117')
    [6] Column
        [0] Row
            [0] Select(name='x', options=['Communications', ...], value='Communications')
            [1] Select(name='y', options=['Min', 'Max', ...], value='Min')
        [1] Row
            [0] Plotly(Figure, name='interactive00134')